![gearshift](img/LOGO_NAME.png)

In [ ]:
import pandas as pd
import shutil
import sys
import subprocess
from io import StringIO
import numpy as np
import ipywidgets as widgets
from tqdm import tqdm
import threading
import queue
import time
import zipfile
import os
from IPython.display import display





# Table definition
tab = widgets.Tab()
# Define the output terminal into application
out = widgets.Output(layout={'border': '1px solid black'})
out.overflow_x = 'scroll'
# Upload file
up = widgets.FileUpload(accept=".xlsx", multiple=False)
sheetNames = widgets.RadioButtons(
    options=['case', 'vehicle', 'engine', 'gearbox_ratios'],
    description='Sheet: ',
    disabled=False)

eraser = widgets.SelectMultiple(
    options=['tab','"'],
    value=['tab'],
    #rows=10,
    description='Eraser: ',
    disabled=False)

rows = widgets.IntSlider(
    value=0,
    step=1,
    description='# of lines:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

button_demo = widgets.Button(
    description='Get demo file',
    disabled=False,
    button_style='warning',
    tooltip='Click to Download demo file',
    icon='arrow-circle-down')

button_demo_empty = widgets.Button(
    description='Empty input file',
    disabled=False,
    button_style='warning',
    tooltip='Click to Download empty demo file',
    icon='arrow-alt-circle-down')

button_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')

button_plot = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='danger',
    tooltip='Click to Plot',
    icon='pencil')

graph_type = widgets.Dropdown(
    options=['Bar Chart', 'Line Chart'],
    value='Bar Chart',
    description='Chart Type:',
    disabled=False)

x_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='X-Axis:',
    disabled=False)

y_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='Y-Axis:',
    disabled=False)

color_picker = widgets.ColorPicker(
    concise=False,
    description='Color Picker: ',
    value='lightblue',
    disabled=False)

toggle = widgets.ToggleButtons(
    options=['Preview  ', 'Info  ', 'Stats  '],
    description='Options',
    disabled=False,
    button_style='warning',
    icons=['search', 'info', 'tachometer'])

button_run = widgets.Button(
    description='Run Gearshift',
    disabled=False,
    button_style='success',
    tooltip='Click to Plot',
    icon='seedling')

accordion = widgets.Accordion(children=[
    widgets.VBox([button_demo]),
    widgets.VBox([button_demo_empty]),
    up, 
    widgets.VBox([sheetNames])])
accordion.set_title(0, 'Get Demo File')
accordion.set_title(1, 'Get Empty File')
accordion.set_title(2, 'File Selection')
accordion.set_title(3, 'Sheet Selection Preview')
accordion_box = widgets.VBox([
    accordion, 
    widgets.HBox([button_preview]),
    out
])

children = [
    accordion_box, 
    widgets.VBox([button_run, out]),
]

tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Run")
#tab.set_title(2, "Download")
tab

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))

def content_reader():
    """
    This function reads and check the format of the input file
    
    :returns:
        - fileName (:py:class:`str`):
            
    """
    if up.value == {}:
        with out:
            print('No Excel file loaded')    
    else:
        fileName = next(iter(up.value))
        fileType = up.value[fileName]['metadata']['type']
        t = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
        
        if fileType == t:
            content = up.value[fileName]['content']
            with open(fileName, 'wb') as f: 
                f.write(content)
                
            return fileName

            
def df_converter():
    fileName = content_reader()
    if fileName is not None:
        input_data = pd.ExcelFile(fileName, engine="openpyxl")
        df = input_data.parse(sheetNames.value)
        numberCases =  input_data.parse("case")["case"].count()
        return df, numberCases - 1
    else:
        return None


def preview():
    df, numberCases = df_converter()
    with out:
        out.clear_output()
        print(f'\n The number of cases that will be run is: {numberCases} \n')
        print(f'\n -----The data in {sheetNames.value} sheet is:----- \n')
        if df is not None:
            display(df)
        else:
            print('Configuration is wrong/missing...')
            

def download():
    with out:
        out.clear_output()
        print('\n -----Downloading demo file:----- \n')
        
        command = !gearshift demo "./output"
        
        for line in command:
            print(line)

def run_command(my_queue):
    
    fileName = content_reader()
    command = !gearshift run "{fileName}" -O "./output"
    my_queue.put(command)
    
def get_empty_demo():
    os.makedirs("./output" or ".", exist_ok=True)
    src = r"../gearshift/demos/gs_input_demo_empty.xlsx"
    dst = "./output"
    with out:
        print('\n -----Downloading Empty Input file:----- \n')
        shutil.copy2(src, dst)
        print('\033[92m'+'\n Done!! you can find it in output folder \n'+'\033[0m')

def runGS():
    
    df, numberCases = df_converter()
    
    with out:
        
        out.clear_output()
        
        my_queue = queue.Queue()
        
        commandGS = threading.Thread(target=run_command, args=(my_queue,))
        
        commandGS.start()
        print('\n -----Executing Gearshift Tool:----- \n')
        
        for i in tqdm(range(numberCases)):
            time.sleep(1.80)
        
        out.clear_output()
        
        command = my_queue.get()
        
        for line in command:
            print(line)
        
        print('\033[92m'+'\n***** Zip file have been created with the all files in the output folder *****\n'+'\033[0m')
        zipf = zipfile.ZipFile('output.zip', 'w', zipfile.ZIP_DEFLATED)
        zipdir('output/', zipf)
        zipf.close()
        
             
def preview_clicked(b):
    preview()
    
def download_demo(b):
    download()

def run_gearshift(b):
    runGS()
    
def download_demo_empty(b):
    get_empty_demo()

button_preview.on_click(preview_clicked)
button_demo.on_click(download_demo)
button_run.on_click(run_gearshift)
button_demo_empty.on_click(download_demo_empty)

display(tab)

### Run with terminal command <a class="anchor" id="terminal"></a>

If you want execute the gearshift tool using the terminal command you can follow the instructions defined in the [documentation](https://gearshift-calculation-tool.readthedocs.io/en/latest/intro.html#quick-start) adding **!** at the beginning of the command:

```
!gearshift run "gs_input_demo.xlsx" -O "./output"
```

To upload the input file you can use the GUI interface to upload 1.2.1.2. Select file for the execution 